In [3]:

import pygame
import math
import time
import random

pygame.init()

clock = pygame.time.Clock()
fps = 60

#set screen dimension
scrn_width, scrn_height = 1200, 800

#create game window
screen = pygame.display.set_mode((scrn_width, scrn_height))
pygame.display.set_caption('santa_game')

#load background images
bg = pygame.image.load("images/sky.png").convert() #sky
bg_width = bg.get_width()
# moon = pygame.image.load("images/moon.png").convert_alpha() #moon
# moon_width = moon.get_width()

#font
pygame.font.init()
my_font = pygame.font.SysFont('Impact', 30)

#define game
bg_scroll = 0
# moon_scroll = 0

tiles = math.ceil(scrn_width / bg_width)
print(tiles)

#create player
dx, dy = 100, 250
player = pygame.Rect(dx, dy, 50, 50)
player_speed = 3
player_speed_diag = math.sqrt((player_speed ** 2)/2)
playertext = 0
playercolor = (255, 0 , 0)

#create chimney
chim_width, chim_height = 100, 300
chimneytext = 0
chimneys = []

chimney_timer = 0
chimney_timer_modifier = 0

chimney_base_speed = 2.5 #pixels per frame
chimney_speed_modifier = 0

chimney_speed_adjusted = 2.5
chimney_gap = random.randrange(4, 7)                 #pixels
chimney_delta = chimney_speed_adjusted / chimney_gap
chimney_spawn_interval = chimney_delta * fps



#create present details
prez_width, prez_height = 30, 30
drop_speed = 5
presents = []
last_drop_time = 0 #tracks last time present was dropped
drop_cooldown = 0.2 #in seconds, cooldown period after drop
presenttext = 0

#create score
score = 0

#game loop
run = True
while run:

    clock.tick(fps)
    #draw scrolling  layers
    for i in range(0, tiles+1):
        screen.blit(bg, (i * bg_width + bg_scroll, 0))

    # for i in range(0, tiles+1):
    #     screen.blit(moon, (i * moon_width + moon_scroll, 0))

    #scroll background layers
    bg_scroll -= 2
    # moon_scroll -= 0.1

    #reset scroll
    if abs(bg_scroll) > bg_width:
        bg_scroll = 0
    # if abs(moon_scroll) > moon_width:
    #     moon_scroll = 0

    #txt
    playertext = my_font.render(f'{round(dx)}, {round(dy)}', False, (255, 255, 255))
    screen.blit(playertext, (0, 0))

    scoretext = my_font.render(f'{score}', False, (255, 255, 255))
    screen.blit(scoretext, (scrn_width-50, 0))

    #draw in player
    pygame.draw.rect(screen, playercolor, player)

    #set keyboard actions
    #player movement
    key = pygame.key.get_pressed()
    # diag left down
    if ((key[pygame.K_a]) or (key[pygame.K_LEFT])) and ((key[pygame.K_s]) or (key[pygame.K_DOWN])):
        player.move_ip(-player_speed_diag, player_speed_diag)
        dx -= player_speed_diag
        dy += player_speed_diag
    # diag left up
    elif ((key[pygame.K_a]) or (key[pygame.K_LEFT])) and ((key[pygame.K_w]) or (key[pygame.K_UP])):
        player.move_ip(-player_speed_diag, -player_speed_diag)
        dx -= player_speed_diag
        dy -= player_speed_diag
    # diag right down
    elif ((key[pygame.K_d]) or (key[pygame.K_RIGHT])) and ((key[pygame.K_s]) or (key[pygame.K_DOWN])):
        player.move_ip(player_speed_diag, player_speed_diag)
        dx += player_speed_diag
        dy += player_speed_diag
    # diag right up
    elif ((key[pygame.K_d]) or (key[pygame.K_RIGHT])) and ((key[pygame.K_w]) or (key[pygame.K_UP])):
        player.move_ip(player_speed_diag, -player_speed_diag)
        dx += player_speed_diag
        dy -= player_speed_diag
    #left
    elif (key[pygame.K_a] == True) or (key[pygame.K_LEFT]) == True:
        player.move_ip(-player_speed, 0)
        dx -= player_speed
    #right
    elif key[pygame.K_d] or (key[pygame.K_RIGHT]) == True:
        player.move_ip(player_speed, 0)
        dx += player_speed
    #up
    elif key[pygame.K_w] or (key[pygame.K_UP]) == True:
        player.move_ip(0, -player_speed)
        dy -= player_speed
    #down
    elif key[pygame.K_s] or (key[pygame.K_DOWN]) == True:
        player.move_ip(0, player_speed)
        dy += player_speed


    # if you get a present in the chimney
    for prez in presents:
        #get coordinates for present corners
        present_coords = {"topleft": [prez["x"], prez["y"]],
                          "topright": [prez["x"]+prez_width, prez["y"]],
                          "bottomleft": [prez["x"], prez["y"]+prez_height],
                          "bottomright": [prez["x"]+prez_width, prez["y"]+prez_height]}
        for chim in chimneys:
            #get coords for chimney corners
            chimney_coords = {"topleft": [chim["x"], chim["y"]],
                          "topright": [chim["x"]+chim['cw'], chim["y"]],
                          "bottomleft": [chim["x"], chim["y"]+chim['ch']],
                          "bottomright": [chim["x"]+chim['cw'], chim["y"]+chim['ch']]}
            #scoring
            corner_count = 0
            for coord in present_coords.values():
                if (coord[0] >= chimney_coords["topleft"][0]) and (coord[0] <= chimney_coords["topright"][0]) and (coord[1] >= chimney_coords["topleft"][1]):
                    corner_count += 1
            if corner_count >= 4:
                score += 1
                presents.remove(prez)
            corner_count = 0

            #right collision (hit right side of present/left of chimney)
            if (present_coords["topright"][0] > chimney_coords['topleft'][0]) and (
                    present_coords['topright'][0] < chimney_coords['topleft'][0]+10) and (
                    present_coords["topright"][1] >= chimney_coords["topleft"][1]):
                prez["speedx"] = chimney_base_speed * 2
            # left collision (hit left side of present/right of chimney)
            if (present_coords["topleft"][0] > chimney_coords['topright'][0]-10) and (
                    present_coords['topleft'][0] < chimney_coords['topright'][0]) and (
                    present_coords["topleft"][1] >= chimney_coords["topright"][1]):
                prez["speedx"] = chimney_base_speed * -0.5
            #bottom right collision (bottom right cornre of prz bounce off top of chim)
            if (present_coords["bottomright"][0] > chimney_coords['topleft'][0]) and (
                    present_coords['bottomright'][0] < chimney_coords['topleft'][0]+20) and (
                    present_coords["bottomright"][1] < chimney_coords["topleft"][1]+20) and (
                    present_coords["bottomright"][1] > chimney_coords["topleft"][1]
            ):
                prez["speedx"] = chimney_base_speed * 2
                prez["speedy"] *= -1
                #prez["speedy"] -= 10

            # bottom left collision (bottom left cornre of prz bounce off top of chim)
            if (present_coords["bottomleft"][0] > chimney_coords['topright'][0]-20) and (
                    present_coords['bottomleft'][0] < chimney_coords['topright'][0]) and (
                    present_coords["bottomleft"][1] < chimney_coords["topright"][1]) and (
                    present_coords["bottomleft"][1] > chimney_coords["topright"][1]-20
            ):
                prez["speedx"] = chimney_base_speed * -0.5
                prez["speedy"] *= -1

                #prez["speedy"] -= 10





            #any prez corner hits outside wall of chim, bounce out
            #when we're 90% inside the top boundaries, it should score
            #should bounce upwards if hits top and doen't go in
            #should bounce sideways and down if hits side



    # for prez in presents:
    #     for chim in chimneys:
    #         if prez.colliderect(chim):
    #             print("poo")




            # if (prez["x"] > chim["x"]) and (prez["x"] < chim["x"] + chimney[2]) and (prez["y"] > chim["y"]) and (
            #         prez["y"] < chim["y"] + chimney[3]):
            #     score += 1
            #     presents.remove(prez)
            # #make it so prez bonks off chimney side
            # elif (prez["x"]+prez_width > chim["x"]) and (prez["x"]+prez_width < chim["x"]+prez_width) and (prez["y"] > chim["y"]) and (prez["y"] < chim["y"] + chimney[3]):
            #     prez["speedx"] = chimney_base_speed*2
            # elif (prez["x"]+prez_width > chim["x"]) and (prez["x"]+prez_width < chim["x"]+prez_width) and (prez["y"]+prez_height > chim["y"]) and (prez["y"]+prez_height < chim["y"] + chimney[3]):
            #     prez["speedx"] = chimney_base_speed*2
            # elif (prez["x"] > chim["x"]) and (prez["x"] < chim["x"] + chimney[2]) and (prez["y"]+prez_height > chim["y"]) and (
            #         prez["y"]+prez_height < chim["y"] + chimney[3]):
            #     score += 1
            #     presents.remove(prez)


    #drop present with spacebar
    current_time = time.time()
    if key[pygame.K_SPACE] & (current_time - last_drop_time > drop_cooldown):
        #add present & its coordinates to list of presents
        presents.append({"x": dx, "y": dy, "speedx": 0, "speedy": 1}) #add present coordinates to list
        last_drop_time = current_time #reset last drop so cooldown can be in effect
    # set present movement
    for prez in presents[:]:
        prez["speedy"] += 0.4
        prez["y"] += prez["speedy"] #add movement speed to coordinate so it moves
        prez["x"] -= prez["speedx"]
        if prez["y"] > scrn_height:
            presents.remove(prez)

    #draw in present rectangle
    for prez in presents:
        pygame.draw.rect(screen, (255, 255, 255), (prez["x"], prez["y"], prez_width, prez_height))

    #spawn chimney
    #need to update chimney timer so that chimneys get spawned faster as score increases, otherwise they appear to be closer togethr
    chimney_timer += 1
    if chimney_timer >= chimney_spawn_interval:
        chimney_height = random.randrange(200, 400)
        chimney_width = random.randrange(80, 150)
        chimneys.append({"ch": chimney_height,
                         "cw": chimney_width,
                         "x": scrn_width,
                         "y": scrn_height-chimney_height})
        chimney_timer = 0
        chimney_gap = random.randrange(3, 9)

    chimney_delta = chimney_gap / chimney_speed_adjusted
    chimney_spawn_interval = chimney_delta * fps

    #move chimney
    for chim in chimneys[:]:
        #gradually increase chimney movement speed
        #every 10 points gives a speed increase of .5
        chimney_speed_modifier = math.floor(score/10)*0.5
        chimney_speed_adjusted = chimney_base_speed + chimney_speed_modifier
        chim["x"] -= chimney_speed_adjusted
        if chim["x"] < -chim["cw"]:
            chimneys.remove(chim)

    #draw in the chimney
    for chim in chimneys:
        pygame.draw.rect(screen, (0, 255, 0), (chim["x"], chim["y"], chim["cw"], chim["ch"]))

    #event handler
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            run = False

    pygame.display.update()

pygame.quit()

2
